In [1]:
from pdf_rag import query_rag

In [2]:
query_rag("what is the plans")

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/pull "HTTP/1.1 200 OK"
c:\1_files\ojt\light_weight_solutions\testing_chroma_db_install_python_10\pdf_rag.py:56: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_db = Chroma(
INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
INFO:root:Loaded existing vector database.
INFO:root:Retriever created.
INFO:root:Chain created successfully.
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:langchain.retrievers.multi_query:Generated queries: ['Here are five different versions of the original user question:', '1.

'According to the document, NetFast offers three internet plans:\n\n1. Basic Plan - $29.99/month - Speed: 50 Mbps - Suitable for browsing and streaming\n2. Standard Plan - $49.99/month - Speed: 150 Mbps - Great for families and work-from-home\n3. Premium Plan - $79.99/month - Speed: 500 Mbps - Best for gaming and ultra-fast streaming'

In [4]:
import json
# Load the JSON file
file_path = "monthly_expenses.json"  # Replace with the correct path to your file
with open(file_path, "r") as file:
    data = json.load(file)

monthly_expenses_json_string = json.dumps(data, indent=4) 

In [8]:
from langchain_ollama import ChatOllama
from langgraph_supervisor import create_supervisor
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

#MODEL_NAME = "qwen2.5:7b-instruct"
#model = ChatOllama(model=MODEL_NAME, temperature=0.0)


load_dotenv()
model = ChatOpenAI(model="gpt-4o", temperature=0.7)
#model = ChatOpenAI(model="gpt-4o")
# Create specialized agents

def add(a: float, b: float) -> float:
    """Add two numbers."""
    return a + b

def multiply(a: float, b: float) -> float:
    """Multiply two numbers."""
    return a * b

def web_search(query: str) -> str:
    """Search the web for information."""
    return (
        "Here are the headcounts for each of the FAANG companies in 2024:\n"
        "1. **Facebook (Meta)**: 67,317 employees.\n"
        "2. **Apple**: 164,000 employees.\n"
        "3. **Amazon**: 1,551,000 employees.\n"
        "4. **Netflix**: 14,000 employees.\n"
        "5. **Google (Alphabet)**: 181,269 employees."
    )

def rag_search(query: str) -> str:
    """Search the information about netfast internet company"""

    rag_result = query_rag(query)
    return (rag_result)

math_agent = create_react_agent(
    model=model,
    tools=[add, multiply],
    name="math_expert",
    prompt="You are a math expert. Always use one tool at a time."
)

research_agent = create_react_agent(
    model=model,
    tools=[web_search],
    name="research_expert",
    prompt="You are a world class researcher with access to web search. Do not do any math."
)

report_agent = create_react_agent(
    model=model,
    tools=[],
    name="report_expert",
    prompt=f"You are a report expert. you can give reports based on this data:{monthly_expenses_json_string }"
)

netfast_agent = create_react_agent(
    model=model,
    tools=[rag_search],
    name="netfast_expert",
    prompt=f"You are a netfast expert. you always use the tool rag_search"
)

# Create supervisor workflow
workflow = create_supervisor(
    [research_agent, math_agent, report_agent, netfast_agent],
    model=model,
    prompt=(
        "You are a team supervisor managing a research expert and a math expert. "
        "For current events, use research_agent. "
        "For math problems, use math_agent."
        "For report queries, use report_agent."
        "For netfast queries, use netfast_agent."
    ),
    output_mode="full_history"
)



In [13]:
model 

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000024225D031C0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000024225E488E0>, root_client=<openai.OpenAI object at 0x0000024225DE2560>, root_async_client=<openai.AsyncOpenAI object at 0x0000024224C52C20>, model_name='gpt-4o', temperature=0.7, model_kwargs={}, openai_api_key=SecretStr('**********'))

In [15]:
# Compile and run
app = workflow.compile()
result = app.invoke({
    "messages": [
        {
            "role": "user",
            "content": "give me the summary of the report, what is the average spending each parameter each year"
        }
    ]
})

In [16]:
for m in result['messages']:
    m.pretty_print()

================================ Human Message =================================

give me the summary of the report, what is the average spending each parameter each year
================================== Ai Message ==================================
Name: supervisor
Tool Calls:
  transfer_to_report_expert (call_PowJ4PnzRnqxaRfU8O2mk8LL)
 Call ID: call_PowJ4PnzRnqxaRfU8O2mk8LL
  Args:
================================= Tool Message =================================
Name: transfer_to_report_expert

Successfully transferred to report_expert
================================== Ai Message ==================================
Name: report_expert

To summarize the report, let's calculate the average spending for each category (Food, Transpo, and Daily) for the years 2023 and 2024.

### 2023:
- **Food**: 
  - Total: 8000 + 7500 + 8500 + 8000 + 7800 + 8200 + 8500 + 8700 + 8100 + 8300 + 8600 + 9000 = 99200
  - Average: 99200 / 12 = 8266.67

- **Transpo**: 
  - Total: 2500 + 2300 + 2600 + 2400 + 25

In [9]:
app = workflow.compile()

# Collect all steps
data = 1
for step in app.stream({
    "messages": [
        {
            "role": "user",
            "content": "how can i code  a python list vs a c# list"
        }
    ]
}):
    print(step) 
    data = step       # Live output
    

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{'supervisor': {'messages': [HumanMessage(content='how can i code  a python list vs a c# list', additional_kwargs={}, response_metadata={}, id='084f8277-3f26-422e-92f5-5a5e7341a00a'), AIMessage(content="Here's a comparison of how lists are coded and used in Python versus C#:\n\n### Python List\n\nIn Python, a list is a built-in data type that can store a collection of items. Lists are dynamic, meaning they can grow and shrink as needed. They are also heterogeneous, meaning they can store items of different types.\n\n```python\n# Creating a list in Python\nmy_list = [1, 2, 3, 4, 5]\n\n# Adding elements\nmy_list.append(6)\n\n# Accessing elements\nfirst_element = my_list[0]  # 1\n\n# Removing elements\nmy_list.remove(3)\n\n# Length of the list\nlength = len(my_list)\n\n# Iterating over the list\nfor item in my_list:\n    print(item)\n```\n\n### C# List\n\nIn C#, a list is part of the `System.Collections.Generic` namespace and is implemented as `List<T>`, where `T` is the type of elements 

In [10]:
for m in data['supervisor']['messages']:
    m.pretty_print()

================================ Human Message =================================

how can i code  a python list vs a c# list
================================== Ai Message ==================================
Name: supervisor

Here's a comparison of how lists are coded and used in Python versus C#:

### Python List

In Python, a list is a built-in data type that can store a collection of items. Lists are dynamic, meaning they can grow and shrink as needed. They are also heterogeneous, meaning they can store items of different types.

```python
# Creating a list in Python
my_list = [1, 2, 3, 4, 5]

# Adding elements
my_list.append(6)

# Accessing elements
first_element = my_list[0]  # 1

# Removing elements
my_list.remove(3)

# Length of the list
length = len(my_list)

# Iterating over the list
for item in my_list:
    print(item)
```

### C# List

In C#, a list is part of the `System.Collections.Generic` namespace and is implemented as `List<T>`, where `T` is the type of elements in the l